In [14]:
!pip install PyPDF2
#Cargar y Preprocesar CVs Transformar los archivos PDF en texto utilizando herramientas como PyPDF2 o pdfminer

from PyPDF2 import PdfReader

def extract_text_from_pdf(file_path):
    reader = PdfReader(file_path)
    text = " ".join(page.extract_text() for page in reader.pages)
    return text


In [13]:
#Limpieza del Texto
import re

def clean_text(text):
    text = re.sub(r'\s+', ' ', text)  # Remover espacios adicionales
    text = re.sub(r'[^\w\s@.-]', '', text)  # Eliminar caracteres especiales
    return text


In [12]:
#Extracción de Información
import spacy

nlp = spacy.load("en_core_web_sm")

def extract_name(text):
    doc = nlp(text)
    for ent in doc.ents:
        if ent.label_ == "PERSON":
            return ent.text
    return None


In [5]:
#Email y Teléfono
#Detectar patrones con expresiones regulares
def extract_email_phone(text):
    email = re.search(r'\b[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Z|a-z]{2,}\b', text)
    phone = re.search(r'\b\d{10}|\+\d{1,3} \d{10}\b', text)
    return email.group(0) if email else None, phone.group(0) if phone else None


In [6]:
# Años de Experiencia Búsqueda de frases relacionadas con experiencia laboral
def extract_experience(text):
    match = re.search(r'(\d+)\s+years of experience', text, re.IGNORECASE)
    return int(match.group(1)) if match else None


In [8]:
# Formación en Inteligencia Artificial
def extract_ai_training(text):
    keywords = ["Artificial Intelligence", "Machine Learning", "Deep Learning", "AI", "IA", "inteligencia artificial"]
    for keyword in keywords:
        if keyword.lower() in text.lower():
            return "Yes"
    return "No"


In [15]:
#Calcular el Score de Confianza
def calculate_score(value):
    return 1.0 if value else 0.0


In [11]:
#Generar JSON de Salida
#Crear un archivo JSON para almacenar los resultados

import json

# Función para procesar un CV y generar el resultado
def process_cv(cv_id, text):
    name = extract_name(text)
    email, phone = extract_email_phone(text)
    experience = extract_experience(text)
    ai_training = extract_ai_training(text)

    # Construir el resultado como un diccionario
    result = {
        "cv_id": cv_id,
        "name": name,
        "email": email,
        "phone": phone,
        "years_of_experience": experience,
        "ai_training": ai_training,
        "scores": {
            "name": calculate_score(name),
            "email": calculate_score(email),
            "phone": calculate_score(phone),
            "years_of_experience": calculate_score(experience),
            "ai_training": calculate_score(ai_training),
        },
    }
    return result

# Probar con muestras de texto
new_samples = [
    "John Doe has 5 years of experience in AI. Contact: johndoe@example.com, Phone: +123456789.",
    "Jane Smith, no AI experience, has 3 years in software. Email: janesmith@domain.com."
]

# Procesar y guardar resultados
all_results = []

for idx, sample in enumerate(new_samples):
    cv_result = process_cv(f"CV_{idx+1}", sample)
    all_results.append(cv_result)

# Guardar todos los resultados en un archivo JSON
with open("output.json", "w") as f:
    json.dump(all_results, f, indent=4)

print("Resultados procesados y guardados en 'output.json'")



Resultados procesados y guardados en 'output.json'


In [16]:
from transformers import pipeline

qa_pipeline = pipeline("question-answering", model="distilbert-base-cased")

def extract_with_llm(text, question):
    return qa_pipeline(question=question, context=text)["answer"]


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/465 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/263M [00:00<?, ?B/s]

Some weights of DistilBertForQuestionAnswering were not initialized from the model checkpoint at distilbert-base-cased and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/436k [00:00<?, ?B/s]

In [17]:
new_samples = [
    "John Doe has 5 years of experience in AI. Contact: johndoe@example.com, Phone: +123456789.",
    "Jane Smith, no AI experience, has 3 years in software. Email: janesmith@domain.com."
]

for idx, sample in enumerate(new_samples):
    name = extract_name(sample)
    email, phone = extract_email_phone(sample)
    experience = extract_experience(sample)
    ai_training = extract_ai_training(sample)
    print(generate_json_output(f"CV_{idx+1}", name, email, phone, experience, ai_training))


{'cv_id': 'CV_1', 'name': 'John Doe', 'email': 'johndoe@example.com', 'phone': None, 'years_of_experience': 5, 'ai_training': 'Yes', 'scores': {'name': 1.0, 'email': 1.0, 'phone': 0.0, 'years_of_experience': 1.0, 'ai_training': 1.0}}
{'cv_id': 'CV_2', 'name': 'Jane Smith', 'email': 'janesmith@domain.com', 'phone': None, 'years_of_experience': None, 'ai_training': 'Yes', 'scores': {'name': 1.0, 'email': 1.0, 'phone': 0.0, 'years_of_experience': 0.0, 'ai_training': 1.0}}
